In [1]:
import geopandas as gpd
import leafmap.foliumap as leafmap
import arcgis 
import requests 
from shapely.geometry import Polygon
import os
import shapely 
import json 

In [2]:
gdf = gpd.read_file("./S123_462bdc6642c74163bc54cd75d385247b_SHP.zip")

In [3]:
#Column name to what it represents 
column_map = {
    "observacio" : "observacion",
    "si_es_una_": "estructura abandonada dentro del mar",
    "venta_const": "venta con construccion",
    "a_ade_come": "comentario adicional",
    "esta_const": "construccion con rotulos pficiales",
    "este_terre": "venta con rotulos oficiales",
    "si_de_casu": "tuvieron problemas con el localizador"
    
}
gdf.columns

Index(['globalid', 'CreationDa', 'Creator', 'EditDate', 'Editor', 'observacio',
       'si_es_una_', 'venta_cont', 'a_ade_come', 'nombre', 'correo_ele',
       'telefono', 'esta_const', 'este_terre', 'si_de_casu', 'pueblo',
       'geometry'],
      dtype='object')

In [4]:
PR_AOI = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -67.35397048193602,
              18.568671638862668
            ],
            [
              -67.35397048193602,
              17.80754468951328
            ],
            [
              -65.11851750203257,
              17.80754468951328
            ],
            [
              -65.11851750203257,
              18.568671638862668
            ],
            [
              -67.35397048193602,
              18.568671638862668
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}


file_list = os.listdir("survey_data")
if len(file_list) > 1:
    raise("Loading gdf error")

#Loads file
file_name = file_list[0]
gdf = gpd.read_file(f"./survey_data/{file_name}")

#spatial filter
aoi_coords = PR_AOI["features"][0]["geometry"]["coordinates"][0]
polygon = Polygon(aoi_coords)
aoi_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon])
filtered_gdf = gpd.sjoin(gdf, aoi_gdf, op='within')

filtered_gdf 

,globalid,CreationDa,Creator,EditDate,Editor,observacio,si_es_una_,venta_cont,a_ade_come,nombre,correo_ele,telefono,esta_const,este_terre,si_de_casu,pueblo,geometry,index_right
0,51f26e1e-7114-49f4-811e-45a3788c7df1,2022-11-16,AmigxsdelMAR,2022-11-16,AmigxsdelMAR,venta,None,Si,None,None,None,0.000000e+00,None,None,None,None,POINT (-65.31222 18.33197),0
1,44c154f2-2631-44d5-8aa0-f9d52c93aa56,2023-03-08,None,2023-03-08,None,Construcción_,None,None,Construcción de residencias de lujo en la zona...,Juan D. Pueblo,hectorj21@gmail.com,1.234568e+08,Sí,None,None,None,POINT (-65.82275 18.41814),0
2,bfa40664-f11a-435c-8946-d60ff811da80,2023-03-09,None,2023-03-09,None,estructura _abandonada,No,None,None,Vanessa,e.vanessauriarte@gmail.com,0.000000e+00,None,None,None,None,POINT (-65.91412 18.44498),0
3,64c4d1c9-e496-4912-834b-65264ac2757a,2023-03-09,None,2023-03-09,None,Cierre o bloqueo de acceso al m,None,None,None,Vanessa,e.vanessauriarte@gmail.com,0.000000e+00,None,None,None,None,POINT (-65.88883 18.44223),0
4,5b28dd1e-a575-4ce5-8ea7-ea014a631223,2023-03-09,None,2023-03-09,None,venta,None,No,None,None,e.vanessauriarte@gmail.com,0.000000e+00,None,Sí,None,None,POINT (-65.83465 18.42799),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2eaa6d0b-75b6-4bcf-80d1-cfd70bfda59d,2023-07-05,None,2023-07-05,None,Cierre o bloqueo de acceso al m,None,None,None,LASchicas de la colmena,Info@amigxsdelmar.org,0.000000e+00,None,None,None,Vieques,POINT (-65.44431 18.14861),0
226,85d23f92-db6f-4ad3-8698-41f00fb870e4,2023-07-05,None,2023-07-05,None,estructura _abandonada,Si,None,"Esta bloqueando la entrada, esta dentro del mar",Las chicas de la colmena,Info@amigxsdelmar.org,0.000000e+00,None,None,None,Vieques,POINT (-65.43870 18.15665),0
227,8a62bb9e-11af-48fc-8b5c-22a177d5195a,2023-07-05,None,2023-07-05,None,Construcción_,None,None,None,Las chicas de la colmena,Info@amigxsdelmar.org,0.000000e+00,No,None,None,Vieques,POINT (-65.43743 18.15742),0
228,fc2aa097-ff84-408b-8eae-c3aaafd26e54,2023-07-08,None,2023-07-08,None,estructura _abandonada,No,None,Hay dos estructuras abandonadas. Una evidentem...,Erimar,erimar.landron3@gmail.com,7.873763e+09,None,None,Aquí enlace de google map\nhttps://maps.app.go...,Aguada,POINT (-67.17081 18.40789),0


In [5]:
def query_pitirre(row):
    # num_records = 10
    url = "http://127.0.0.1:8000/parcels/"

    x, y = row.x, row.y
    # x,y = -67.16076297678497, 18.450717761715765
    params = {
        "point": f"{x},{y}",
        "dist": 15
    }

    #TODO: Get authorization from env file
    headers = {
    'Authorization': 'Basic YWRtaW46MTIzNA=='
    }

    response = requests.get(url, params=params, headers=headers)
    
    return response.json()["results"]

# owner_gdf = gpd.GeoDataFrame()
filtered_gdf["payload"] = filtered_gdf["geometry"].apply(query_pitirre)

In [18]:
filtered_gdf.iloc[0]["payload"]["features"][0]["properties"]["owner_name"]

'PEÑA GARCIA COSME '

In [23]:
owners = []
for i in range(len(filtered_gdf)):
    payload = filtered_gdf.iloc[i]["payload"]["features"]
    for j in range(len(payload)):
        owners.append(payload[j]["properties"]["owner_name"])
    # break 


In [27]:
with open("./owners.txt", 'w') as f:
    f.write(str(owners))

In [5]:
terrestre_gdf = gpd.read_file("g11_conserv_areas_naturales_protegidas_terrestres_2019(1).zip")
print(terrestre_gdf)

       fid                                     names  \
0      1.0             Bosque del Pueblo de Adjuntas   
1      2.0              Bosque Estatal de Cambalache   
2      3.0               Bosque Estatal de Cerrillos   
3      4.0               Bosque Estatal de Guajataca   
4      5.0                 Bosque Estatal de Guánica   
..     ...                                       ...   
171  172.0            Zona de Conservación del Carso   
172  173.0         Reserva Natural Laguna Tortuguero   
173  174.0    Área Natural Protegida Río Sana Muerto   
174  175.0        Área Natural Protegida Ojo de Agua   
175  176.0  Reserva Natural de Isla de Mona y Monito   

                                                  mgmt  \
0    Co-manejo: DRNA y Taller de Arte y Cultura de ...   
1    Departamento de Recursos Naturales y Ambiental...   
2    Departamento de Recursos Naturales y Ambiental...   
3    Departamento de Recursos Naturales y Ambiental...   
4    Departamento de Recursos Natural

In [6]:
marina_gdf = gpd.read_file("g11_conserv_areas_naturales_protegidas_marinas_2019.zip")
marina_gdf

,names,mgmt,ownership,year_estab,loc_desig,terr_mar,gis_source,notes,geometry
0,Ext. Marina Bosque Estatal de Guánica,Departamento de Recursos Naturales y Ambiental...,Dominio público,1985,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((150562.668 212809.719, 150562.676 21..."
1,Ext. Marina del Bosque Estatal de Boquerón,Departamento de Recursos Naturales y Ambiental...,Dominio público,1998,Extensión Marina,Marina,DRNA 2015?mayo,None,"MULTIPOLYGON (((121355.765 214771.323, 121350...."
2,Ext. Marina R.N. Caño La Boquilla,Departamento de Recursos Naturales y Ambiental...,Dominio público,2002,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((121693.914 243697.073, 101477.601 24..."
3,Ext. Marina R.N. Punta Guaniquilla,Departamento de Recursos Naturales y Ambiental...,Dominio público,2002,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((115317.817 226278.038, 118875.189 22..."
4,Ext. Marina Reserva Natural de Isla de Mona y ...,Departamento de Recursos Naturales y Ambiental...,Dominio público,1986,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((65595.457 237602.784, 65883.519 2368..."
5,Ext. Marina Reserva Natural Hacienda La Esperanza,Departamento de Recursos Naturales y Ambiental...,Dominio público,1987,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((192250.650 271248.616, 192250.644 27..."
6,Ext. Marina Reserva Natural La Parguera,Departamento de Recursos Naturales y Ambiental...,Dominio público,1986,Extensión Marina,Marina,DRNA 2015?mayo,None,"MULTIPOLYGON (((138385.025 215783.455, 138379...."
7,Ext. Marina R.N. Cueva del Indio,Departamento de Recursos Naturales y Ambiental...,Dominio público,1992,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((178735.446 273008.733, 178732.821 27..."
8,Ext. Marina R.N. Finca Belvedere,Departamento de Recursos Naturales y Ambiental...,Dominio público,2003,Extensión Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((108017.947 229029.287, 112782.976 22..."
9,Reserva Natural Arrecifes de la Cordillera,Departamento de Recursos Naturales y Ambiental...,Dominio público,1980,Reserva Natural Marina,Marina,DRNA 2015?mayo,None,"POLYGON ((302816.002 259138.339, 307582.432 25..."


In [7]:
m = leafmap.Map()
m.add_gdf(terrestre_gdf)
m.add_gdf(marina_gdf)
# m.add_point()
m